In [38]:
import pandas as pd
import numpy as np
import pickle as pkl
import xgboost as xgb

In [63]:
results = pkl.load(open("../results/ist3/model_selection.pkl", "rb"))

In [64]:
for learner, learner_results in results.items():
    print(learner)
    
    for sec in ["if_pehe", "pseudo_outcome_r", "pseudo_outcome_dr"]:
        
        print(learner, sec, np.mean(learner_results[sec]))
    

XLearner
XLearner if_pehe 10.283402302764674
XLearner pseudo_outcome_r 0.04579735719617524
XLearner pseudo_outcome_dr 0.09190736354001233
DRLearner
DRLearner if_pehe 17.25073258461358
DRLearner pseudo_outcome_r 0.05943752400029999
DRLearner pseudo_outcome_dr 0.11905753622188002
SLearner
SLearner if_pehe 6.603826351319571
SLearner pseudo_outcome_r 0.0367111590868117
SLearner pseudo_outcome_dr 0.0736431013658913
TLearner
TLearner if_pehe 25.775143487561955
TLearner pseudo_outcome_r 0.0717606396018427
TLearner pseudo_outcome_dr 0.14359767171729088
